In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('../../../data_set/train-data.csv', index_col=0)

df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [67]:
df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

Controllo della variabile '**Mileage**'. Essa è espressa come stringa, nel formato '\<valore\> \<unità di misura\>'

In [68]:
df['Mileage']

0       26.6 km/kg
1       19.67 kmpl
2        18.2 kmpl
3       20.77 kmpl
4        15.2 kmpl
           ...    
6014     28.4 kmpl
6015     24.4 kmpl
6016     14.0 kmpl
6017     18.9 kmpl
6018    25.44 kmpl
Name: Mileage, Length: 6019, dtype: object

Controlliamo se ci sono differenti unità di misura:

In [69]:
mileage_units = df['Mileage'].str.split().str[-1]
mileage_units.unique()

array(['km/kg', 'kmpl', nan], dtype=object)

Notiamo che abbiamo due diverse unità 'km/kg' e 'kmpl'.

Vediamo per quante osservaioni appaiono le varie unità:

In [70]:
mileage_units.value_counts()

Mileage
kmpl     5951
km/kg      66
Name: count, dtype: int64

Abbiamo visto che non ci sono errori o unità di misura fittizie, ci sono solamente alcuni valori mancanti. Vediamo quanti sono:

In [71]:
mileage_units.isna().sum()

np.int64(2)

Abbiamo solo due valori mancanti che gestiremo in seguito.

Inoltre, notiamo che i vari valori di essa utilizzano differenti unità di misura. Questo accade perché tipicamente, per calcolare la quantità carburante consumato al chilometro, si utilizzano propreità diverse in base al tipo carburante dell'auto.

In genere:
- kmpl - chilometri al litro, usato per carburanti liquidi (benzina, diesel, GPL)
- km/kg - chilometri al chilogrammo, usato per carburanti diversi (CNG)

Vorremmo quindi, rimuovere o riempire i valori mancanti e sopratutto standardizzare le differenti unità di misura, in modo da averne solamente una.

Importante: ogni tipo di carburante ha una differente formula di conversione.

In [72]:
df_2 = df[df['Mileage'].isna()]
df_2

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
4446,Mahindra E Verito D4,Chennai,2016,50000,Electric,Automatic,First,NaN,72 CC,41 bhp,5.0,13.58 Lakh,13.00
4904,Toyota Prius 2009-2016 Z4,Mumbai,2011,44000,Electric,Automatic,First,NaN,1798 CC,73 bhp,5.0,NaN,12.75


Vediamo se ci sono valori insensati:

In [76]:
df_copy = df.copy()

df_copy['Mileage'] = df_copy['Mileage'].str.split(' ').str[0]
df_copy['Mileage'] = pd.to_numeric(df_copy['Mileage'], errors='coerce')

# Fattori di conversione da km/l a km/kg
conversion_factors = {
    'Petrol': 1/0.74,   # approssimativo
    'Diesel': 1/0.85,   # approssimativo
    'LPG': 1/0.51,      # approssimativo
    'CNG': 1,            # già in km/kg
    'Electric': np.nan   # non convertibile
}

# Funzione di conversione per riga
def convert_kmpl_to_kmpkg(row):
    fuel = row['Fuel_Type']
    mileage = row['Mileage']
    # Se il valore non è valido o carburante non convertibile
    if pd.isna(mileage) or mileage <= 0 or fuel not in conversion_factors:
        return np.nan
    factor = conversion_factors[fuel]
    return mileage / factor

# Applico la conversione
df_copy['Mileage_kmpkg'] = df_copy.apply(convert_kmpl_to_kmpkg, axis=1)

# Calcolo la mediana dei valori validi
median_value = df_copy['Mileage_kmpkg'].median()

# Riempio i valori NaN (inclusi quelli inizialmente NaN o <=0)
df_copy['Mileage_kmpkg'] = df_copy['Mileage_kmpkg'].fillna(median_value)

df_copy['Mileage'] = df_copy['Mileage_kmpkg']

df_copy.drop(columns=['Mileage_kmpkg'], inplace=True)

In [77]:
bad_values = df_copy.loc[df_copy['Mileage'] <= 0]
print(bad_values.value_counts())

Series([], Name: count, dtype: int64)


Convertiamo

| Fuel     | Potere calorifico | Conversione (L → kg) approssimativa       |
| -------- | ----------------- | ----------------------------------------- |
| Petrol   | 32 MJ/l           | 1 L ≈ 0.74 kg (32/43 MJ/kg)               |
| Diesel   | 36 MJ/l           | 1 L ≈ 0.85 kg (36/42 MJ/kg)               |
| LPG      | 25 MJ/l           | 1 L ≈ 0.51 kg (25/49 MJ/kg)               |
| CNG      | 50 MJ/kg          | 1 L ≈ ??? non serve, lo lasciamo in km/kg |
| Electric | -                 | non convertibile                          |
